# 走行の録画(データセット用のデータの作成)

2つのカメラを同時に起動し、データセット用のカメラ画像を保存します。

### Boardを判定

In [1]:
import Jetson.GPIO as GPIO

BOARD_NAME = GPIO.gpio_pin_data.get_data()[0]

mode_descriptions = {
    "JETSON_NX": ["15W_2CORE", "15W_4CORE", "15W_6CORE", "10W_2CORE", "10W_4CORE"],
    "JETSON_XAVIER": ["MAXN", "MODE_10W", "MODE_15W", "MODE_30W"],
    "JETSON_NANO": ["MAXN", "5W"],
    "JETSON_ORIN": ["MAXN", "MODE_15W", "MODE_30W", "MODE_40W"],
    "JETSON_ORIN_NANO": ["MODE_15W", "MODE_7W"]
}

product_names = {
    "JETSON_NX": "Jetson Xavier NX",
    "JETSON_XAVIER": "Jetson AGX Xavier",
    "JETSON_NANO": "Jetson Nano",
    "JETSON_ORIN": "Jetson AGX Orin",
    "JETSON_ORIN_NANO": "Jetson Orin Nano"
}

# ボードごとのI2Cバス番号と初期Powerモードを定義する
board_settings = {
    "JETSON_NX": (8, 3),
    "JETSON_XAVIER": (8, 2),
    "JETSON_NANO": (1, 0),
    "JETSON_ORIN": (7, 0),
    "JETSON_ORIN_NANO": (7, 0)
}

i2c_busnum, power_mode = board_settings.get(BOARD_NAME, (None, None))
mode_description = mode_descriptions.get(BOARD_NAME, [])
product_name = product_names.get(BOARD_NAME, "未知のボード")

if power_mode is not None and power_mode < len(mode_description):
    mode_str = mode_description[power_mode]
    print("------------------------------------------------------------")
    print(f"{product_name}を認識: I2Cバス番号: {i2c_busnum}, Powerモード: {mode_str}({power_mode})に設定します。")
    print("------------------------------------------------------------")
else:
    print("未知のボードまたは不正なモードです。")

------------------------------------------------------------
Jetson Orin Nanoを認識: I2Cバス番号: 7, Powerモード: MODE_15W(0)に設定します。
------------------------------------------------------------


In [2]:
if (product_name == "Jetson Orin Nano") or (product_name == "Jetson AGX Orin"):
    print("Docker起動のため電力モードは変更できません。")
else:
    !echo "jetson" | sudo -S nvpmodel -m $power_mode

Docker起動のため電力モードは変更できません。


In [3]:
!echo "jetson" | sudo -S nvpmodel -q

[sudo] password for jetson: NV Power Mode: 15W
0


In [4]:
if (product_name == "Jetson Orin Nano") or (product_name == "Jetson AGX Orin"):
    print("Docker起動のためjetson_clocksは起動できません。")
else:
    !echo "jetson" | sudo -S jetson_clocks

Docker起動のためjetson_clocksは起動できません。


In [5]:
!i2cdetect -y -r $i2c_busnum

     0  1  2  3  4  5  6  7  8  9  a  b  c  d  e  f
00:          -- -- -- -- -- 08 -- -- -- -- -- -- -- 
10: -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 
20: -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 
30: -- -- -- -- -- -- -- -- -- -- -- -- 3c -- -- -- 
40: 40 -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 
50: -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 
60: -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 
70: 70 -- -- -- -- -- -- --                         


In [6]:
!ls /dev/video*

/dev/video0  /dev/video1


In [7]:
from jetcam.csi_camera import CSICamera
from jetcam.utils import bgr8_to_jpeg
#from csi_camera import CSICamera
#from utils import bgr8_to_jpeg

if (product_name == "Jetson Orin Nano") or (product_name == "Jetson AGX Orin"):
    cam0 = CSICamera(capture_device=0,width=224, height=224, capture_fps=30)
    cam1 = CSICamera(capture_device=1,width=224, height=224, capture_fps=30)
else:
    cam0 = CSICamera(capture_device=0,width=224, height=224, capture_fps=30)
    cam1 = CSICamera(capture_device=1,width=224, height=224, capture_fps=30)

ModuleNotFoundError: No module named 'csi_camera'

In [ ]:
cam0.running = False
cam1.running = False

In [8]:
import ipywidgets
from ipywidgets import Button, Layout, Textarea, HBox, VBox, Label
import subprocess
import os
import cv2
import time
import threading
import smbus
import json

i2c = smbus.SMBus(i2c_busnum)
addr = 0x08
count = 0
current_path = os.getcwd()

In [10]:
def map_rc(x, in_min, in_max, out_min, out_max):
    return (x - in_min) * (out_max - out_min) // (in_max - in_min) + out_min

def live():
    global count,running,DIR,i2c,cam0,cam1,cv2,annotation,num,current_path
    
    save_xy_dir0 = os.path.join(current_path, task_widget.value, dataset_widget.value+"_cam0", "xy")
    save_xy_dir1 = os.path.join(current_path, task_widget.value, dataset_widget.value+"_cam1", "xy")
    save_speed_dir0 = os.path.join(current_path, task_widget.value, dataset_widget.value+"_cam0", "speed")
    save_speed_dir1 = os.path.join(current_path, task_widget.value, dataset_widget.value+"_cam1", "speed")

    if not os.path.exists(save_xy_dir0):
        subprocess.call(['mkdir', '-p', save_xy_dir0])
    if not os.path.exists(save_xy_dir1):
        subprocess.call(['mkdir', '-p', save_xy_dir1])

    if not os.path.exists(save_speed_dir0):
        subprocess.call(['mkdir', '-p', save_speed_dir0])
    if not os.path.exists(save_speed_dir1):
        subprocess.call(['mkdir', '-p', save_speed_dir1])

    write_log("録画を開始しました。")
    count = 0
    num = 0
    start_time = time.time()
    while running:
        img0 = cam0.read()
        img1 = cam1.read()
        if annotation == True:
            try:
                data = i2c.read_i2c_block_data(addr, 0x01, 12)
                xy = data[0] << 24 | data[1] << 16 | data[2] << 8 | data[3]
                xy = map_rc(xy, left, right, 0, 224)
                if xy < 0:
                    xy = 0
                elif xy > 224:
                    xy = 224
                xy_img_name = "{}_{}_{:0=5}.jpg".format(xy, 112, count)
                xy_image_path0 = os.path.join(save_xy_dir0, xy_img_name)
                cv2.imwrite(xy_image_path0, img0)
                xy_image_path1 = os.path.join(save_xy_dir1, xy_img_name)
                cv2.imwrite(xy_image_path1, img1)
                speed = data[4] << 24 | data[5] << 16 | data[6] << 8 | data[7]
                speed = map_rc(speed, stop, front, 0, 224)
                if speed < 0:
                    speed = 0
                elif speed > 224:
                    speed = 224
                speed_img_name = "{}_{}_{:0=5}.jpg".format(0, speed, count)
                speed_image_path0 = os.path.join(save_speed_dir0, speed_img_name)
                cv2.imwrite(speed_image_path0, img0)
                speed_image_path1 = os.path.join(save_speed_dir1, speed_img_name)
                cv2.imwrite(speed_image_path1, img1)
                
                
                num+=1
                if time.time() - start_time > 3.0:
                    fps = num / 3.0
                    write_log(f"FPS: {fps:.1f} (3秒ごとに表示)")
                    start_time = time.time()
                    fps = 0
                    num = 0
                count+=1
            except Exception as e:
                write_log(f"An error occurred: {e}")
        else:
            #if count % 2 == 0:
            img_name = "{}_{}_{:0=5}.jpg".format(0,0,count)
            image_path0 = os.path.join(save_xy_dir0, img_name)
            cv2.imwrite(image_path0, img0)
            image_path1 = os.path.join(save_xy_dir1, img_name)
            cv2.imwrite(image_path1, img1)
            num+=1
            if time.time() - start_time > 3.0:
                fps = num / 3.0
                write_log(f"FPS: {fps:.1f} (3秒ごとに表示)")
                start_time = time.time()
                fps = 0
                num = 0
            count+=1
    
def record(change):
    global running, execute_thread
    if dataset_widget.value == "":
        write_log("保存先 dataset名を指定してください。")
    else:
        running = True
        execute_thread = threading.Thread(target=live)
        execute_thread.start()

def stop_record(change):
    global count, running, execute_thread, num
    running = False
    execute_thread.join()
    write_log(str(count) + "枚の画像を生成し、録画を終了しました。")

In [11]:
try:
    with open('raw_params.json') as f:
        json_str = json.load(f)

        stop = json_str["raw_speed"]["stop"]
        front = json_str["raw_speed"]["front"]
        back = json_str["raw_speed"]["back"]
        left = json_str["raw_steering"]["left"]
        center = json_str["raw_steering"]["center"]
        right = json_str["raw_steering"]["right"]
except:
    print("Don't exit raw_param.json")

Don't exit raw_param.json


In [12]:
l = Layout(flex='0 1 auto', height='100px', min_height='100px', width='auto')
process_widget = ipywidgets.Textarea(description='ログ', value='', layout=l)

process_no = 0
def write_log(msg):
    global process_widget, process_no
    process_no = process_no + 1
    process_widget.value = str(process_no) + ": " + msg + "\n" + process_widget.value

In [13]:
CATEGORIES = ['xy','speed']
TASK = ['camera']
DIR = "./"

category_widget = ipywidgets.Dropdown(options=CATEGORIES, description='category')
dataset_widget = ipywidgets.Text(description='dataset')
task_widget = ipywidgets.Dropdown(options=TASK, description='task')
start_button = ipywidgets.Button(description='Record')
stop_button = ipywidgets.Button(description='Stop')
annotation_box = ipywidgets.Checkbox(False, description='Auto Annotation')

start_button.on_click(record)
stop_button.on_click(stop_record)

annotation = False
def on_annotation(c):
    global annotation
    annotation ^= True

annotation_box.observe(on_annotation)

In [14]:
import time

release_button = ipywidgets.Button(description='Camera開放')


def stop_camera(c):
    global cam0,cam1
    cam0.running = False
    time.sleep(1)
    cam0.cap.release()
    write_log("カメラ0を開放しました。")
    cam1.running = False
    time.sleep(1)
    cam1.cap.release()
    write_log("カメラ1を開放しました。")

release_button.on_click(stop_camera)

In [15]:
separator = ipywidgets.HTML('<hr style="border-color:gray;margin:10px 0"/>')
title1 = ipywidgets.HTML('<b>【1.カメラの録画】</b><br>データセット用の走行データを保存できます。走行データはcameraフォルダにデータが保存されます。Auto Annotationにチェックをいれると操作の値も保存できます。JetRacerでは、Auto Annotationでつけたデータセットの学習はうまくいきません。参考値を取得する用途でのみ使用してください。')
title2 = ipywidgets.HTML('<b>【2.カメラの開放処理】</b><br>走行データの収集が完了したらカメラの開放処理を実行します。')

data_collection_widget = ipywidgets.VBox([
    separator,
    title1,
    task_widget,
    dataset_widget,
    annotation_box,
    ipywidgets.HBox([start_button, stop_button]),
    process_widget,
    separator,
    title2,
    release_button,
    process_widget
])

display(data_collection_widget)